# SISSO Demonstration: Symbolic Regression with Uncertainty Quantification

This notebook demonstrates **SISSO (Sure Independence Screening and Sparsifying Operator)** for discovering interpretable analytical expressions from data with calibrated uncertainty estimates.

## What is SISSO?

SISSO is a symbolic regression method that:
- Discovers **interpretable equations** (not black-box models)
- Uses **feature construction** to build candidate expressions
- Applies **sparsifying operators** to select the best terms
- Provides **calibrated uncertainty** via the hat matrix method

**Key Features of this wrapper:**
- sklearn-compatible API (fit/predict/score)
- Uncertainty quantification with `return_std=True`
- Works with cross-validation and pipelines

**Reference:** TorchSISSO - https://arxiv.org/abs/2410.01752

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.base import clone

from pycse.sklearn import SISSO

# Set style
plt.style.use("default")
plt.rcParams["figure.dpi"] = 100

np.random.seed(42)
print("Imports successful!")

## 1. Discovering a Simple Equation

Let's generate data from a known equation and see if SISSO can recover it.

**True function:** y = 2*x0 + 3*x0*x1 - 1.5

In [ ]:
# True function
def true_function(X):
    return 2 * X[:, 0] + 3 * X[:, 0] * X[:, 1] - 1.5


# Generate training data
n_train = 100
X_train = np.random.rand(n_train, 2) * 2 - 1  # [-1, 1]
noise = 0.1 * np.random.randn(n_train)
y_train = true_function(X_train) + noise

# Generate test data
n_test = 50
X_test = np.random.rand(n_test, 2) * 2 - 1
y_test = true_function(X_test) + 0.1 * np.random.randn(n_test)

print(f"Training samples: {n_train}")
print(f"Test samples: {n_test}")
print("True equation: y = 2*x0 + 3*x0*x1 - 1.5")

## 2. Fit SISSO Model

We use:
- `n_expansion=2` to allow feature combinations like `x0*x1`
- `n_term=2` to find a 2-term equation
- Custom feature names for readability

In [ ]:
model = SISSO(
    operators=["+", "-", "*", "/"], n_expansion=2, n_term=2, k=20, feature_names=["x0", "x1"]
)

model.fit(X_train, y_train)

print(f"Discovered equation: {model.equation_}")
print(f"Training R-squared: {model.r2_:.4f}")
print(f"Training RMSE: {model.rmse_:.4f}")

## 3. Predictions with Uncertainty Quantification

The `return_std=True` argument returns calibrated prediction uncertainties computed using the hat matrix method.

### Theory

Since SISSO's final model is a **linear combination of nonlinear features**:
```
y = c0 + c1*f1(x) + c2*f2(x) + ... + ct*ft(x)
```

We can use standard linear regression UQ via the **hat matrix**:
- Residual variance: sigma^2 = RSS / (n - p)
- Prediction variance: Var(y*) = sigma^2 * (1 + phi*^T (Phi^T Phi)^-1 phi*)

In [ ]:
y_pred, y_std = model.predict(X_test, return_std=True)

# Evaluate on test set
test_r2 = r2_score(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Test R-squared: {test_r2:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Mean uncertainty: {np.mean(y_std):.4f}")
print(f"Residual std (sigma): {model.sigma_:.4f}")
print(f"Calibration factor: {model.calibration_factor_:.4f}")

## 4. Visualize Predictions vs True Values

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Parity plot
ax = axes[0]
ax.errorbar(
    y_test, y_pred, yerr=2 * y_std, fmt="o", alpha=0.6, capsize=3, label="Predictions +/- 2 sigma"
)
ax.plot(
    [y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--", label="Perfect prediction"
)
ax.set_xlabel("True values")
ax.set_ylabel("Predicted values")
ax.set_title("Parity Plot with Uncertainty")
ax.legend()
ax.grid(True, alpha=0.3)

# Residuals with uncertainty bands
ax = axes[1]
residuals = y_test - y_pred
ax.errorbar(y_pred, residuals, yerr=2 * y_std, fmt="o", alpha=0.6, capsize=3)
ax.axhline(0, color="k", linestyle="--")
ax.set_xlabel("Predicted values")
ax.set_ylabel("Residuals")
ax.set_title("Residuals with Uncertainty Bands")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Checking Uncertainty Calibration

For well-calibrated uncertainties:
- ~68% of true values should fall within +/-1 sigma of predictions
- ~95% should fall within +/-2 sigma

In [ ]:
z_scores = (y_test - y_pred) / y_std

coverage_1sigma = np.mean(np.abs(z_scores) < 1) * 100
coverage_2sigma = np.mean(np.abs(z_scores) < 2) * 100

print(f"Coverage within +/-1 sigma: {coverage_1sigma:.1f}% (expected ~68%)")
print(f"Coverage within +/-2 sigma: {coverage_2sigma:.1f}% (expected ~95%)")

# Plot z-score distribution
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(z_scores, bins=15, density=True, alpha=0.7, label="Observed z-scores")

# Overlay standard normal
x_norm = np.linspace(-4, 4, 100)
y_norm = np.exp(-(x_norm**2) / 2) / np.sqrt(2 * np.pi)
ax.plot(x_norm, y_norm, "r-", linewidth=2, label="Standard normal")

ax.set_xlabel("Z-score")
ax.set_ylabel("Density")
ax.set_title("Calibration Check: Z-score Distribution")
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 6. Extrapolation: Uncertainty Should Increase

When predicting outside the training domain, uncertainties should grow to reflect the model's reduced confidence.

In [ ]:
# In-distribution points
X_in = np.random.rand(100, 2) * 2 - 1  # Same [-1, 1] range as training
_, std_in = model.predict(X_in, return_std=True)

# Out-of-distribution points (extrapolation)
X_out = np.random.rand(100, 2) * 2 + 2  # [2, 4] range (outside training)
_, std_out = model.predict(X_out, return_std=True)

print(f"Mean uncertainty (in-distribution): {np.mean(std_in):.4f}")
print(f"Mean uncertainty (extrapolation): {np.mean(std_out):.4f}")
print(f"Ratio: {np.mean(std_out) / np.mean(std_in):.2f}x")

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(std_in, bins=20, alpha=0.6, label="In-distribution", density=True)
ax.hist(std_out, bins=20, alpha=0.6, label="Extrapolation", density=True)
ax.set_xlabel("Prediction uncertainty (sigma)")
ax.set_ylabel("Density")
ax.set_title("Uncertainty Distribution: In-domain vs Extrapolation")
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 7. Gaps in Training Data: Interpolation with UQ

A key test of uncertainty quantification is how the model handles **gaps in training data**.
The uncertainty should increase in regions where we have no training data.

We'll test three functions with gaps, providing polynomial basis features where needed:
1. **Linear**: y = 2x (SISSO can discover directly)
2. **Quadratic**: y = 3x^2 + 1 (provide x^2 as feature)
3. **Sine**: y = sin(x) (use polynomial approximation with x, x^2, x^3)

In [ ]:
def generate_data_with_gap(func, x_range, gap_start, gap_end, n_samples=100, noise_std=0.1):
    """Generate 1D data with a gap in the middle."""
    n_left = n_samples // 2
    n_right = n_samples - n_left

    x_left = np.linspace(x_range[0], gap_start, n_left)
    x_right = np.linspace(gap_end, x_range[1], n_right)
    x_train = np.concatenate([x_left, x_right])

    y_true = func(x_train)
    noise = noise_std * np.random.randn(len(x_train))
    y_train = y_true + noise

    return x_train, y_train


# Define three test cases
datasets = {
    "Linear: y = 2x": {
        "func": lambda x: 2 * x,
        "x_range": (0, 1),
        "gap": (0.35, 0.65),
        "noise": 0.1,
        "make_features": lambda x: x.reshape(-1, 1),  # Just x
        "feature_names": ["x"],
        "n_term": 1,
        "n_expansion": 1,
    },
    "Quadratic: y = 3x^2 + 1": {
        "func": lambda x: 3 * x**2 + 1,
        "x_range": (-1, 1),
        "gap": (-0.3, 0.3),
        "noise": 0.15,
        "make_features": lambda x: np.column_stack([x, x**2]),  # x and x^2
        "feature_names": ["x", "x_sq"],
        "n_term": 1,
        "n_expansion": 1,
    },
    "Sine: y = sin(x)": {
        "func": np.sin,
        "x_range": (0, 2 * np.pi),
        "gap": (2.5, 4.0),
        "noise": 0.1,
        "make_features": lambda x: np.column_stack([x, x**2, x**3]),  # Polynomial basis
        "feature_names": ["x", "x2", "x3"],
        "n_term": 3,
        "n_expansion": 1,
    },
}

print("Generated datasets with gaps:")
for name, params in datasets.items():
    print(f"  - {name}: gap at [{params['gap'][0]:.2f}, {params['gap'][1]:.2f}]")

In [ ]:
# Fit SISSO models and plot results
fig, axes = plt.subplots(3, 1, figsize=(8, 12))

for idx, (name, params) in enumerate(datasets.items()):
    ax = axes[idx]

    # Generate data with gap
    np.random.seed(42)
    x_train, y_train = generate_data_with_gap(
        params["func"],
        params["x_range"],
        params["gap"][0],
        params["gap"][1],
        n_samples=100,
        noise_std=params["noise"],
    )

    # Create features
    X_train = params["make_features"](x_train)

    # Fit SISSO
    model = SISSO(
        operators=["+", "-", "*", "/"],
        n_expansion=params["n_expansion"],
        n_term=params["n_term"],
        k=20,
        feature_names=params["feature_names"],
    )
    model.fit(X_train, y_train)

    # Create dense prediction grid
    x_plot = np.linspace(params["x_range"][0], params["x_range"][1], 200)
    X_plot = params["make_features"](x_plot)

    # Predict with uncertainty
    y_pred, y_std = model.predict(X_plot, return_std=True)

    # True function for comparison
    y_true = params["func"](x_plot)

    # Calculate MAE
    y_pred_train = model.predict(X_train)
    mae = np.mean(np.abs(y_train - y_pred_train))

    # Plot uncertainty band
    ax.fill_between(
        x_plot, y_pred - 2 * y_std, y_pred + 2 * y_std, alpha=0.3, color="red", label="95% CI"
    )

    # Plot prediction
    ax.plot(x_plot, y_pred, "r-", linewidth=2, label="SISSO prediction")

    # Plot true function
    ax.plot(x_plot, y_true, "gray", linewidth=1, linestyle="--", alpha=0.5, label="True function")

    # Plot training data
    ax.scatter(x_train, y_train, c="dimgray", s=30, alpha=0.6, marker="s", label="Training data")

    # Shade gap region
    ax.axvspan(params["gap"][0], params["gap"][1], alpha=0.1, color="purple")

    # Add MAE annotation
    ax.text(
        0.02,
        0.98,
        f"MAE={mae:.3f}",
        transform=ax.transAxes,
        fontsize=10,
        verticalalignment="top",
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
    )

    ax.set_xlabel("x")
    ax.set_ylabel("y")
    eq_display = model.equation_[:50] + "..." if len(model.equation_) > 50 else model.equation_
    ax.set_title(f"{name}\nEquation: {eq_display}")
    ax.legend(loc="best", fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNote: The purple shaded region shows the gap where no training data exists.")
print("Uncertainty bands reflect the model's confidence based on the discovered equation.")

## 8. Detailed Gap Analysis

Let's quantify how uncertainty behaves in different regions.

In [ ]:
# Analyze the quadratic case in detail
np.random.seed(42)
params = datasets["Quadratic: y = 3x^2 + 1"]

x_train, y_train = generate_data_with_gap(
    params["func"],
    params["x_range"],
    params["gap"][0],
    params["gap"][1],
    n_samples=100,
    noise_std=params["noise"],
)

X_train = params["make_features"](x_train)

model = SISSO(
    operators=["+", "-", "*", "/"], n_expansion=1, n_term=1, k=20, feature_names=["x", "x_sq"]
)
model.fit(X_train, y_train)

# Sample points inside gap, outside gap, and extrapolation
x_in_gap = np.linspace(-0.25, 0.25, 50)
x_outside_gap = np.concatenate([np.linspace(-0.9, -0.4, 25), np.linspace(0.4, 0.9, 25)])
x_extrap = np.concatenate([np.linspace(-2, -1.1, 25), np.linspace(1.1, 2, 25)])

_, std_in_gap = model.predict(params["make_features"](x_in_gap), return_std=True)
_, std_outside_gap = model.predict(params["make_features"](x_outside_gap), return_std=True)
_, std_extrap = model.predict(params["make_features"](x_extrap), return_std=True)

print("Quadratic function uncertainty analysis:")
print(f"  Discovered equation: {model.equation_}")
print("\nMean uncertainty by region:")
print(f"  In gap [-0.3, 0.3]:     {np.mean(std_in_gap):.4f}")
print(f"  Training region:        {np.mean(std_outside_gap):.4f}")
print(f"  Extrapolation:          {np.mean(std_extrap):.4f}")
print("\nRatios:")
print(f"  Extrapolation / Training: {np.mean(std_extrap) / np.mean(std_outside_gap):.2f}x")

## 9. Example: Discovering a Physical Law

Let's try to discover the equation for kinetic energy:

**E = 0.5 * m * v^2**

In [ ]:
# Generate physics data
n = 100
mass = np.random.uniform(1, 10, n)  # kg
velocity = np.random.uniform(0, 20, n)  # m/s
energy = 0.5 * mass * velocity**2  # True kinetic energy
energy_noisy = energy + 0.05 * energy * np.random.randn(n)  # 5% noise

# Include m, v, and v^2 as features
X_physics = np.column_stack([mass, velocity, velocity**2])
y_physics = energy_noisy

# Fit SISSO
model_physics = SISSO(
    operators=["+", "-", "*", "/"], n_expansion=1, n_term=1, k=20, feature_names=["m", "v", "v_sq"]
)
model_physics.fit(X_physics, y_physics)

print("True equation: E = 0.5 * m * v^2")
print(f"Discovered: {model_physics.equation_}")
print(f"R-squared: {model_physics.r2_:.4f}")

## 10. sklearn Compatibility

SISSO works with sklearn's model selection and pipeline tools.

In [ ]:
# Cross-validation on 2D data
np.random.seed(42)
X_cv = np.random.rand(100, 2)
y_cv = X_cv[:, 0] + X_cv[:, 1] + 0.1 * np.random.randn(100)

model_cv = SISSO(n_expansion=1, n_term=1, feature_names=["x0", "x1"])
scores = cross_val_score(model_cv, X_cv, y_cv, cv=3, scoring="r2")
print(f"Cross-validation R-squared scores: {scores}")
print(f"Mean CV R-squared: {scores.mean():.4f} +/- {scores.std():.4f}")

# Clone test
cloned = clone(model)
print("\nCloned model parameters:")
print(f"  n_expansion={cloned.n_expansion}")
print(f"  n_term={cloned.n_term}")
print(f"  k={cloned.k}")

# Get/set params
params = model.get_params()
print(f"\nAll parameters: {params}")

## 11. Summary

The `SISSO` estimator provides:

1. **Interpretable equations** - Discovers symbolic expressions, not black boxes
2. **sklearn compatibility** - Works with cross-validation, pipelines, etc.
3. **Calibrated uncertainty** - Hat matrix method with empirical calibration
4. **Extrapolation awareness** - Uncertainty grows outside training domain

### Key Parameters

| Parameter | Description |
|-----------|-------------|
| `operators` | List of operators: `['+', '-', '*', '/']`, optionally `'exp'`, `'ln'`, etc. |
| `n_expansion` | Feature expansion depth (2-3 typical) |
| `n_term` | Number of terms in equation (1-3 typical) |
| `k` | Features kept per screening iteration |
| `feature_names` | Names for interpretable equations |

### Tips for Feature Engineering

SISSO discovers equations as linear combinations of constructed features. For best results:
- With multiple features (x0, x1), SISSO can discover products like x0*x1
- For polynomial terms (x^2, x^3), provide them explicitly as features
- Use meaningful feature names for interpretable equations

### Usage Pattern

```python
from pycse.sklearn import SISSO

# Fit model
model = SISSO(
    operators=['+', '-', '*', '/'],
    n_expansion=2,
    n_term=2,
    feature_names=['x', 'y', 'z']
)
model.fit(X_train, y_train)

# Get discovered equation
print(model.equation_)

# Predict with uncertainty
y_pred, y_std = model.predict(X_test, return_std=True)
```

### Installation

```bash
pip install pycse[sisso]
# or
pip install TorchSisso
```

### References

- [TorchSISSO Paper](https://arxiv.org/abs/2410.01752)
- [TorchSISSO GitHub](https://github.com/PaulsonLab/TorchSISSO)